# En este notebook corrijo los errores del spam_ham_dataset_modeloNB_pipeline_mejoras.ipynb 

Dado los resultados aunque buenos decido aplicar SVM al dataset en inglés porque da menos error en los Reporte de los ham incorrectamente etiquetados como spam

In [2]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk


# Función de stemming
stemmer = SnowballStemmer("english")
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in word_tokenize(text)])

# Cargar el dataset modificado
df_modificado = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')


# Aplicar stemming
df_modificado['text'] = df_modificado['text'].apply(stem_text)

# Dividir el dataset en conjunto de entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(df_modificado['text'], df_modificado['label_num'], test_size=0.2, random_state=42, stratify=df_modificado['label_num'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Crear y entrenar el pipeline con bigramas y stemming, usando SMOTE
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
    ('smote', SMOTE(random_state=42)),
    ('classifier', MultinomialNB())
])
pipeline.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación
y_val_pred = pipeline.predict(X_val)
print("Reporte de Clasificación en el conjunto de validación:")
print(classification_report(y_val, y_val_pred, target_names=['ham', 'spam']))
print("Matriz de Confusión en el conjunto de validación:")
print(confusion_matrix(y_val, y_val_pred))
print("Exactitud en el conjunto de validación:")
print(accuracy_score(y_val, y_val_pred))

# Guardar el modelo y el vectorizador
joblib.dump(pipeline, '../modelos_y_vectorizadores_en/naive_bayes_smote_pipeline_improved.pkl')

# Evaluar el modelo en el conjunto de prueba
y_test_pred = pipeline.predict(X_test)
print("Reporte de Clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred, target_names=['ham', 'spam']))
print("Matriz de Confusión en el conjunto de prueba:")
print(confusion_matrix(y_test, y_test_pred))
print("Exactitud en el conjunto de prueba:")
print(accuracy_score(y_test, y_test_pred))

# Verifica la longitud de las predicciones y datos de prueba
print("Longitud de X_test:", len(X_test))
print("Longitud de y_test:", len(y_test))
print("Longitud de y_test_pred:", len(y_test_pred))

# Alinear los índices para crear el DataFrame de resultados
results_df = pd.DataFrame({
    'text': X_test.reset_index(drop=True),  # Resetear índices para asegurar alineación
    'actual': y_test.reset_index(drop=True).map({0: 'ham', 1: 'spam'}),
    'predicted': pd.Series(y_test_pred).map({0: 'ham', 1: 'spam'})
})

# Identificar los casos donde el modelo se ha equivocado
incorrect_cases = results_df[results_df['actual'] != results_df['predicted']]

# Mostrar los casos incorrectos
print("Casos donde el modelo se ha equivocado:")
print(incorrect_cases)


Reporte de Clasificación en el conjunto de validación:
              precision    recall  f1-score   support

         ham       1.00      0.96      0.98       707
        spam       0.91      0.99      0.95       292

    accuracy                           0.97       999
   macro avg       0.95      0.98      0.96       999
weighted avg       0.97      0.97      0.97       999

Matriz de Confusión en el conjunto de validación:
[[679  28]
 [  2 290]]
Exactitud en el conjunto de validación:
0.96996996996997
Reporte de Clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

         ham       1.00      0.95      0.97       706
        spam       0.88      0.99      0.93       293

    accuracy                           0.96       999
   macro avg       0.94      0.97      0.95       999
weighted avg       0.96      0.96      0.96       999

Matriz de Confusión en el conjunto de prueba:
[[668  38]
 [  3 290]]
Exactitud en el conjunto de prueba:
0.958

Esto es sin entrenar el modelo con todo train y validacion

Los resultados del modelo Naive Bayes (NB) en el conjunto de prueba son bastante buenos, indicando un rendimiento sólido en la clasificación de mensajes como "ham" (no deseado) o "spam" (deseado). Aquí está un análisis detallado de estos resultados:

### Reporte de Clasificación

#### Clase "ham":
- **Precisión (Precision)**: 1.00 - El modelo predice correctamente todas las instancias etiquetadas como "ham". No hay falsos positivos.
- **Recall (Sensibilidad)**: 0.95 - El modelo identifica correctamente el 95% de las instancias reales de "ham".
- **F1-score**: 0.97 - La media armónica de precisión y recall es alta, indicando un excelente rendimiento en esta clase.
- **Support (Soporte)**: 706 - El número de instancias de "ham" en el conjunto de prueba.

#### Clase "spam":
- **Precisión (Precision)**: 0.88 - El modelo predice correctamente el 88% de las instancias etiquetadas como "spam".
- **Recall (Sensibilidad)**: 0.99 - El modelo identifica correctamente el 99% de las instancias reales de "spam".
- **F1-score**: 0.93 - La media armónica de precisión y recall es buena, aunque no tan alta como para "ham".
- **Support (Soporte)**: 293 - El número de instancias de "spam" en el conjunto de prueba.

#### Totales:
- **Accuracy (Precisión global)**: 0.96 - El modelo clasifica correctamente el 96% de las instancias en el conjunto de prueba.
- **Macro avg** (Promedio macro):
  - **Precision**: 0.94
  - **Recall**: 0.97
  - **F1-score**: 0.95
- **Weighted avg** (Promedio ponderado):
  - **Precision**: 0.96
  - **Recall**: 0.96
  - **F1-score**: 0.96

### Matriz de Confusión

#### Interpretación:
- **668 verdaderos negativos (ham correctamente clasificado)**
- **38 falsos positivos (ham incorrectamente clasificado como spam)**
- **3 falsos negativos (spam incorrectamente clasificado como ham)**
- **290 verdaderos positivos (spam correctamente clasificado)**

### Análisis Detallado

1. **Precisión y Recall**:
   - La **precisión** para "ham" es perfecta, lo que significa que todos los mensajes predichos como "ham" son realmente "ham". Para "spam", la precisión es alta pero no perfecta, indicando que hay algunos mensajes predichos como "spam" que no lo son.
   - El **recall** es muy alto para ambas clases, especialmente para "spam", lo que significa que casi todos los mensajes de "spam" reales son identificados correctamente.

2. **F1-Score**:
   - Los altos valores de F1-score para ambas clases indican que el modelo tiene un buen equilibrio entre precisión y recall. La ligera diferencia entre las clases refleja que la clasificación de "spam" es un poco más difícil para el modelo en términos de precisión.

3. **Accuracy**:
   - Una precisión global del 96% sugiere que el modelo tiene un rendimiento muy alto en general, con solo un pequeño porcentaje de errores.

4. **Matriz de Confusión**:
   - La matriz de confusión muestra que el modelo comete muy pocos errores, con solo 38 falsos positivos y 3 falsos negativos. Esto refuerza la idea de que el modelo es particularmente efectivo en identificar "spam" con muy pocos mensajes de "spam" clasificados incorrectamente como "ham".

### Conclusión

El modelo Naive Bayes muestra un buen rendimiento en el conjunto de prueba, especialmente en términos de recall para ambas clases y precisión para la clase "ham". Aunque la precisión para "spam" es ligeramente inferior, sigue siendo alta, y el bajo número de falsos negativos indica que el modelo es muy eficaz para identificar correctamente los mensajes de "spam". En general, el modelo proporciona una clasificación confiable con una precisión global del 96%, lo que lo convierte en una herramienta valiosa para la clasificación de mensajes de "ham" y "spam".